Imports

In [ ]:
#imports
from tqdm import tqdm
from transformers import pipeline

import sys
import os

utils_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'utils'))

if utils_path not in sys.path:
    sys.path.insert(0, utils_path)

from llm_extractor_utils import make_multi_prompt, llm_extraction_multi_openai
from general_utils import load_data, prepare_all_samples, calculate_metrics

Data Loading

In [ ]:
# Load data
df = load_data("../data/training_dataset_synthetic.csv")
print(f"Loaded {len(df)} records")

In [ ]:
#Inspect df
df.head()

In [ ]:
# Prepare all samples
samples = prepare_all_samples(df)
print(f"Prepared {len(samples)} samples")
#samples[0]

LLM Evaluation

In [ ]:
#Choose whether to use HuggingFace 'hf' or OpenAI 'openai'
#Note if choosing OpenAI need to set OPENAI_API_KEY in .env file
method = 'openai' #options are 'hf' or 'openai'

In [ ]:
# Define LLM to use 
# If using HF any text generation model from HuggingFace can be used, see: https://huggingface.co/models?pipeline_tag=text-generation)
# If using OpenAI you can use any model from OpenAI provided it supports text generation, see: https://platform.openai.com/docs/models
model = 'gpt-4o'

In [ ]:
# Define generator if using HF option
#generator = pipeline("text-generation", model=model, device=-1)

In [ ]:
#Prompt to use
prompt_to_use = 'multi_prompt.txt'

In [ ]:
#Process all samples, make prompt, do llm extraction and make prediction
predictions = []

for sample in tqdm(samples, desc="Processing notes"):
    # Create multi-extraction prompt
    prompt = make_multi_prompt(
        note_text=sample['note_text'],
        prompt_filename=prompt_to_use,
        entities_list=sample['entities_list'], #optinal - does not need to be passed
        dates=sample['dates'] #optional - does not need to be passed
    )
    
    # Get all relationships in one call
    relationships = llm_extraction_multi_openai(prompt, model=model)
    
    # Add to predictions
    predictions.extend(relationships)

In [ ]:
#Look at prediction
predictions

In [ ]:
#Calculcate metrics
metrics = calculate_metrics(predictions, df)
metrics